In [ ]:
import tensorflow as tf
import numpy as np

#데이터 다운로드
from keras.datasets import cifar10

(train_x_list, train_y_list), (test_x, test_y) = cifar10.load_data()

class Model():
    def __init__(self,sess,name):
        self.sess=sess
        self.name=name
        self._build_net()
    

    def _build_net(self):
        with tf.variable_scope(self.name):
            self.X=tf.placeholder(tf.float32,[None,32,32,3])
            self.Y=tf.placeholder(tf.int32,[None,1])
            Y_one_hot=tf.one_hot(self.Y,10)
            Y_one_hot=tf.reshape(Y_one_hot,[-1,10])
            self.keep_prob=tf.placeholder(tf.float32)
            self.training=tf.placeholder(tf.bool)

            W1=tf.get_variable("W1",shape=[3,3,3,32],initializer=tf.contrib.layers.xavier_initializer())
            L1=tf.nn.conv2d(self.X,W1,strides=[1,1,1,1],padding='SAME')
            L1=tf.nn.relu(tf.layers.batch_normalization(L1, training=self.training))
            L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L1=tf.nn.dropout(L1,keep_prob=self.keep_prob)

            W2=tf.get_variable("W2",shape=[3,3,32,64],initializer=tf.contrib.layers.xavier_initializer())
            L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
            L2=tf.nn.relu(tf.layers.batch_normalization(L2, training=self.training))
            L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L2=tf.nn.dropout(L2,keep_prob=self.keep_prob)
            
            W3=tf.get_variable("W3",shape=[3,3,64,128],initializer=tf.contrib.layers.xavier_initializer())
            L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
            L3=tf.nn.relu(tf.layers.batch_normalization(L3, training=self.training))
            L3=tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L3=tf.nn.dropout(L3,keep_prob=self.keep_prob)
            
            W4=tf.get_variable("W4",shape=[3,3,128,256],initializer=tf.contrib.layers.xavier_initializer())
            L4=tf.nn.conv2d(L3,W4,strides=[1,1,1,1],padding='SAME')
            L4=tf.nn.relu(tf.layers.batch_normalization(L4, training=self.training))
            L4=tf.nn.dropout(L4,keep_prob=self.keep_prob)
            L4=tf.reshape(L4,[-1,4*4*256])

            W5=tf.get_variable("W5",shape=[4*4*256,625],initializer=tf.contrib.layers.xavier_initializer())
            B1=tf.Variable(tf.random_uniform([625]))
            L5=tf.nn.relu(tf.matmul(L4,W5)+B1)
            L5=tf.nn.dropout(L5,keep_prob=self.keep_prob)
            
            W6=tf.get_variable("W6",shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
            B2=tf.Variable(tf.random_normal([10]))
            
            self.logits=tf.matmul(L5,W6)+B2
            self.cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=Y_one_hot))
            self.optimizer=tf.train.AdamOptimizer(0.01).minimize(self.cost)

            correct_prediction=tf.equal(tf.argmax(self.logits,1),tf.argmax(Y_one_hot,1))
            self.accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
            
    def predict(self,x_test,keep_prob=1.0,training=False):
        return self.sess.run(self.logits,feed_dict={self.X:x_test,self.keep_prob:keep_prob,self.training:training})
    
    def get_accuracy(self,x_test,y_test,keep_prob=1.0,training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.keep_prob:keep_prob,self.training:training})
    def train(self,x_data,y_data,keep_prob=0.7,training=True):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data,self.keep_prob:keep_prob,self.training:True})

sess=tf.Session()
m=[]
for i in range(2):
    m.append(Model(sess,"model"+str(i)))

training_epoch=1000
batch_size=100
prediction=np.zeros(len(test_y)*10).reshape(len(test_y),10)
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost_list=np.zeros(len(m))
    total_batch=500
    idx=0
    for i in range(total_batch):
        batch_xs,batch_ys=train_x_list[idx:idx+batch_size,:],train_y_list[idx:idx+batch_size,:]
        idx+=100
        for m_idx,mm in enumerate(m):
            c,_=mm.train(batch_xs,batch_ys)
            avg_cost_list[m_idx]+=c/total_batch
    print("epoch: %04d" %(epoch+1),"cost: ",avg_cost_list)
for m_idx,mm in enumerate(m):
    print("Index: ",m_idx," Accuracy:",mm.get_accuracy(test_x,test_y))
    p=mm.predict(test_x)
    prediction+=p
ensemble_correct_prediction=tf.equal(tf.argmax(prediction,1),test_y)
ensemble_accuracy=tf.reduce_mean(tf.cast(ensemble_correct_prediction,tf.float32))
print('ensemble accuracy:',sess.run(ensemble_accuracy))